In [1]:
# Install required packages
# pip install transformers pandas scikit-learn pytorch tqdm

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json
import logging
import re
import math
from collections import Counter

2025-04-07 18:48:11.216525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('merged_dataset.csv')


In [3]:
df.head(
)

,Unnamed: 0,id,chords,release_date,genres,decade,rock_genre,artist_id,main_genre,spotify_song_id,spotify_artist_id,artist_name,song_title,estimated_key,roman_chords,lyrics
0,1,2,<intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs...,2003-01-01,'alternative metal' 'alternative rock' 'nu met...,2000.0,pop rock,artist_2,metal,2ffJZ2r8HxI5DHcmf3BO6c,694QW15WkebjcrWgQHzRYF,Everclear,I Want To Die A Beautiful Death,C,<intro_1> III II VI/Is III II VI/Is <verse_1> ...,Everybody here in this small town\nLooks used ...
1,2,3,<intro_1> Csmin <verse_1> A Csmin A Csmin A Cs...,2003-01-01,'alternative metal' 'canadian rock' 'funk meta...,2000.0,canadian rock,artist_3,metal,5KiY8SZEnvCPyIEkFGRR3y,0niJkG4tKkne3zwr7I8n9n,Finger Eleven,Sad Exchange,C,<intro_1> Ismin <verse_1> VI Ismin VI Ismin VI...,Quietly thinking to myself\nSharing half our m...
2,3,4,<intro_1> D Dmaj7 D Dmaj7 <verse_1> Emin A D G...,2022-09-23,NaN,2020.0,NaN,artist_4,NaN,01TtAcUqyLCRBZq4ZZiQWS,17BfKBemmMGO5ZAK25wraW,RAPHA,Funny,D,<intro_1> I Imaj7 I Imaj7 <verse_1> IImin V I ...,NaN
3,4,5,<intro_1> C <verse_1> G C G C <chorus_1> F Dmi...,2023-02-10,'modern country pop',2020.0,NaN,artist_5,pop,3zUecdrWC3IqrNSjhnoF3G,4GGfAshSkqoxpZdoaHm7ky,ERNEST,Anything But Sober,Gm,<intro_1> iv <verse_1> i iv i iv <chorus_1> bV...,NaN
4,5,6,<intro_1> C G C G <verse_1> C G C G C Bmin Emi...,2018-08-24,'classic opm' 'opm',2010.0,NaN,artist_6,NaN,2zGr6pFE273K7nnu8ACKjk,33kC1w8s5cXEi1zyJZuqSI,Paolo Santos,Come On Home,G,<intro_1> IV I IV I <verse_1> IV I IV I IV III...,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440134 entries, 0 to 440133
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         440134 non-null  int64  
 1   id                 440134 non-null  int64  
 2   chords             440134 non-null  object 
 3   release_date       422034 non-null  object 
 4   genres             376364 non-null  object 
 5   decade             422034 non-null  float64
 6   rock_genre         130596 non-null  object 
 7   artist_id          440134 non-null  object 
 8   main_genre         313653 non-null  object 
 9   spotify_song_id    440134 non-null  object 
 10  spotify_artist_id  440134 non-null  object 
 11  artist_name        440134 non-null  object 
 12  song_title         440134 non-null  object 
 13  estimated_key      440134 non-null  object 
 14  roman_chords       440134 non-null  object 
 15  lyrics             284637 non-null  object 
dtypes:

In [5]:
df.isna().sum()

Unnamed: 0                0
id                        0
chords                    0
release_date          18100
genres                63770
decade                18100
rock_genre           309538
artist_id                 0
main_genre           126481
spotify_song_id           0
spotify_artist_id         0
artist_name               0
song_title                0
estimated_key             0
roman_chords              0
lyrics               155497
dtype: int64

In [6]:
df.dropna(subset=['lyrics'],inplace=True)

In [7]:
def extract_chords_by_section(chord_sequence):
    """
    Extract chords divided by section from a chord sequence string.
    
    Args:
        chord_sequence (str): The full chord progression string with section markers
            Example: "<intro_1> C <verse_1> F C E7 Amin C F C G7 C F C G7"
    
    Returns:
        dict: A dictionary where keys are section names and values are lists of chords
            Example: {
                "intro_1": ["C"],
                "verse_1": ["F", "C", "E7", "Amin", "C", "F", "C", "G7", "C", "F", "C", "G7"]
            }
    """
    if not isinstance(chord_sequence, str) or not chord_sequence.strip():
        return {}
    
    sections = {}
    current_section = None
    current_chords = []
    
    # Split the chord sequence into tokens
    tokens = chord_sequence.split()
    
    # Regular expression to identify section markers like <verse_1>
    import re
    section_pattern = re.compile(r'<([^>]+)>')
    
    for token in tokens:
        # Check if this token is a section marker
        section_match = section_pattern.match(token)
        
        if section_match:
            # Save the previous section if it exists
            if current_section and current_chords:
                sections[current_section] = current_chords
            
            # Start a new section
            current_section = section_match.group(1)
            current_chords = []
        else:
            # Add this chord to the current section
            current_chords.append(token)
    
    # Add the last section
    if current_section and current_chords:
        sections[current_section] = current_chords
    
    return sections

# Example usage:
# If you have a pandas DataFrame with a 'chords' column
df['chords_by_section'] = df['chords'].apply(extract_chords_by_section)

In [8]:
df['chords_by_section']

0         {'intro_1': ['E', 'D', 'A/Cs', 'E', 'D', 'A/Cs...
1         {'intro_1': ['Csmin'], 'verse_1': ['A', 'Csmin...
8         {'chorus_1': ['Amin', 'G', 'F', 'G', 'Amin', '...
11        {'intro_1': ['D', 'G', 'D', 'G', 'D', 'G', 'D'...
12        {'intro_1': ['E', 'G', 'D', 'A'], 'verse_1': [...
                                ...                        
440096    {'intro_1': ['D', 'G', 'D', 'G', 'D', 'G', 'D'...
440100    {'intro_1': ['C', 'C7', 'C', 'C7'], 'verse_1':...
440103    {'intro_1': ['Eno3d', 'Ano3d', 'Eno3d', 'B', '...
440129    {'intro_1': ['Eb/G', 'Abmaj7', 'Bbadd9', 'Eb',...
440130    {'verse_1': ['Bb', 'C', 'Bb', 'Amin', 'Bb', 'A...
Name: chords_by_section, Length: 284637, dtype: object

In [9]:
def normalize_sharp_notation(chord_sequence):
    """
    Normalize chord notation by converting all 's' sharp indicators to '#'.
    
    For example:
    - 'Cs' becomes 'C#'
    - 'Fs' becomes 'F#'
    - 'Gs' becomes 'G#'
    
    Args:
        chord_sequence (str or dict): Either a full chord sequence string or a dictionary 
                                      of sections with chord lists (from extract_chords_by_section)
    
    Returns:
        Same type as input: The normalized chord sequence or dictionary with normalized chords
    """
    import re
    
    # Pattern to match a note with 's' sharp indicator
    # Looks for any of the notes A-G followed by 's', optionally followed by additional characters
    sharp_pattern = re.compile(r'([A-G])s([a-zA-Z0-9/]*)')
    
    if isinstance(chord_sequence, str):
        # For string input, replace directly in the string
        normalized = re.sub(sharp_pattern, r'\1#\2', chord_sequence)
        return normalized
    
    elif isinstance(chord_sequence, dict):
        # For dictionary input (from extract_chords_by_section)
        normalized_dict = {}
        
        for section, chords in chord_sequence.items():
            normalized_chords = []
            
            for chord in chords:
                # Apply the replacement to each chord
                normalized_chord = re.sub(sharp_pattern, r'\1#\2', chord)
                normalized_chords.append(normalized_chord)
            
            normalized_dict[section] = normalized_chords
        
        return normalized_dict
    
    else:
        # Return unchanged if input is neither string nor dict
        return chord_sequence


# Example usage with a string:
# chord_str = "<intro_1> Cs <verse_1> F Cs Gs7 Amin Cs F Cs As7"
# normalized_str = normalize_sharp_notation(chord_str)
# print(normalized_str)
# Output: "<intro_1> C# <verse_1> F C# G#7 Amin C# F C# A#7"

# Example usage with a dictionary from extract_chords_by_section:
# sections_dict = extract_chords_by_section(chord_str)
# normalized_dict = normalize_sharp_notation(sections_dict)
# print(normalized_dict)
# Output: {'intro_1': ['C#'], 'verse_1': ['F', 'C#', 'G#7', 'Amin', 'C#', 'F', 'C#', 'A#7']}

df['chords_by_section'] = df['chords_by_section'].apply(normalize_sharp_notation)

In [10]:
df['roman_chords_by_section'] = df['roman_chords'].apply(extract_chords_by_section)
df['roman_chords_by_section'] = df['roman_chords_by_section'].apply(normalize_sharp_notation)
df['roman_chords_by_section']

0         {'intro_1': ['III', 'II', 'VI/Is', 'III', 'II'...
1         {'intro_1': ['Ismin'], 'verse_1': ['VI', 'Ismi...
8         {'chorus_1': ['imin', 'bVII', 'bVI', 'bVII', '...
11        {'intro_1': ['III', 'VI', 'III', 'VI', 'III', ...
12        {'intro_1': ['V', 'bVII', 'IV', 'I'], 'verse_1...
                                ...                        
440096    {'intro_1': ['V', 'I', 'V', 'I', 'V', 'I', 'V'...
440100    {'intro_1': ['I', 'I7', 'I', 'I7'], 'verse_1':...
440103    {'intro_1': ['Ino3d', 'IVno3d', 'Ino3d', 'V', ...
440129    {'intro_1': ['bVII/ii', 'bIIImaj7', 'ivadd9', ...
440130    {'verse_1': ['bII', 'bIII', 'bII', 'imin', 'bI...
Name: roman_chords_by_section, Length: 284637, dtype: object

In [11]:
class ChordProgressionGenerator(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=6, feature_dim=1):
        super().__init__()
        
        # Text prompt encoder (uses pre-trained model)
        self.text_encoder = AutoModel.from_pretrained("distilbert-base-uncased")
        self.feature_dim = feature_dim
        # Freeze text encoder to preserve pre-trained knowledge
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        
        # Additional features encoder (genre, decade, etc.)
        self.feature_encoder = nn.Linear(feature_dim, d_model)
        
        # Decoder for chord progression generation
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead), 
            num_layers=num_layers
        )
        
        # Output projection to chord vocabulary
        self.output_projection = nn.Linear(d_model, vocab_size)
        
    def forward(self, prompt_ids, prompt_mask, features, tgt_chords, tgt_mask):
        # Encode text prompt
        prompt_encoding = self.text_encoder(prompt_ids, attention_mask=prompt_mask).last_hidden_state
        
        # Encode additional features
        feature_encoding = self.feature_encoder(features)
        
        # Combine encodings
        encoder_output = torch.cat([prompt_encoding, feature_encoding], dim=1)
        
        # Generate chord progression
        decoder_output = self.decoder(tgt_chords, encoder_output, tgt_mask=tgt_mask)
        
        # Project to vocabulary
        logits = self.output_projection(decoder_output)
        
        return logits

In [12]:
# Create the chord tokenizer to convert chords to token IDs
class ChordTokenizer:
    def __init__(self):
        self.chord_to_id = {"<PAD>": 0, "<BOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.id_to_chord = {v: k for k, v in self.chord_to_id.items()}
        self.section_pattern = re.compile(r'<([^>]+)>')
        
    def fit(self, chord_sequences):
        """Build vocabulary from chord sequences."""
        all_chords = set()
        for seq in chord_sequences:
            if isinstance(seq, str):
                # Split by space and filter out section markers
                chords = [c for c in seq.split() if not self.section_pattern.match(c)]
                all_chords.update(chords)
        
        # Add all unique chords to vocabulary
        for i, chord in enumerate(sorted(all_chords)):
            idx = len(self.chord_to_id)
            self.chord_to_id[chord] = idx
            self.id_to_chord[idx] = chord
            
        print(f"Created chord vocabulary with {len(self.chord_to_id)} tokens")
        return self
    
    def tokenize(self, chord_sequence, max_length=50, add_special_tokens=True):
        """Convert chord sequence to token IDs."""
        if not isinstance(chord_sequence, str) or not chord_sequence.strip():
            return [1, 2] + [0] * (max_length - 2) if add_special_tokens else [0] * max_length
        
        # Initialize with BOS token if needed
        tokens = [1] if add_special_tokens else []
        
        # Process each chord
        chords = chord_sequence.split()
        for chord in chords:
            # Check if it's a section marker
            if self.section_pattern.match(chord):
                continue  # Skip section markers for now
            
            # Add chord token
            token_id = self.chord_to_id.get(chord, self.chord_to_id["<UNK>"])
            tokens.append(token_id)
        
        # Add EOS token if needed
        if add_special_tokens:
            tokens.append(2)
        
        # Pad or truncate to max_length
        if len(tokens) < max_length:
            tokens += [0] * (max_length - len(tokens))
        else:
            tokens = tokens[:max_length-1] + ([2] if add_special_tokens else [])
        
        return tokens
    
    def decode(self, token_ids):
        """Convert token IDs back to chord sequence."""
        chords = []
        for token_id in token_ids:
            # Skip special tokens
            if token_id in [0, 1, 2]:
                continue
            
            chord = self.id_to_chord.get(token_id, "<UNK>")
            chords.append(chord)
        
        return " ".join(chords)

In [13]:
# Create the PyTorch dataset for lyrics-to-chord generation
class LyricsChordDataset(Dataset):
    def __init__(self, df, text_tokenizer, chord_tokenizer, max_lyrics_length=128, max_chord_length=50):
        self.df = df
        self.text_tokenizer = text_tokenizer
        self.chord_tokenizer = chord_tokenizer
        self.max_lyrics_length = max_lyrics_length
        self.max_chord_length = max_chord_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Get lyrics
        lyrics = row['lyrics'] if not pd.isna(row['lyrics']) else ""
            
        # Tokenize the lyrics
        lyrics_encoding = self.text_tokenizer(
            lyrics,
            padding="max_length",
            truncation=True,
            max_length=self.max_lyrics_length,
            return_tensors="pt"
        )
        
        # Get chord tokens - using the original chords column for simplicity 
        # (could also use normalized_chords if you created that column)
        chord_sequence = row['roman_chords']
        chord_tokens = torch.tensor(
            self.chord_tokenizer.tokenize(chord_sequence, self.max_chord_length)
        )
        
        # No genre features, just lyrics-to-chord mapping
        return {
            "prompt_ids": lyrics_encoding["input_ids"].flatten(),
            "prompt_attention_mask": lyrics_encoding["attention_mask"].flatten(),
            "chord_tokens": chord_tokens,
        }

In [14]:
# Positional encoding for transformer models
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [15]:
# Updated model architecture for lyrics-to-chord generation
class LyricsToChordModel(nn.Module):
    def __init__(self, chord_vocab_size, text_model_name="distilbert-base-uncased", 
                 d_model=256, nhead=8, num_encoder_layers=6, num_decoder_layers=6):
        super().__init__()
        
        # Text encoder (frozen pre-trained model)
        self.text_encoder = AutoModel.from_pretrained(text_model_name)
        
        # Freeze the text encoder weights
        for param in self.text_encoder.parameters():
            param.requires_grad = False
            
        # Text projection
        self.text_projection = nn.Linear(self.text_encoder.config.hidden_size, d_model)
        
        # Chord embedding
        self.chord_embedding = nn.Embedding(chord_vocab_size, d_model, padding_idx=0)
        
        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, max_len=150)
        
        # Transformer
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=d_model * 4,
            dropout=0.1
        )
        
        # Output projection
        self.output_projection = nn.Linear(d_model, chord_vocab_size)
        
        # Save hyperparameters
        self.d_model = d_model
        self.chord_vocab_size = chord_vocab_size

    def encode(self, prompt_ids, prompt_attention_mask):
        """Encode the input lyrics."""
        # Process text prompt
        text_outputs = self.text_encoder(
            input_ids=prompt_ids,
            attention_mask=prompt_attention_mask
        )
        text_hidden = text_outputs.last_hidden_state
        
        # Project text hidden states
        src = self.text_projection(text_hidden)
        
        # Add positional encoding
        src = self.pos_encoder(src)
        
        # Create source mask for padding tokens
        src_key_padding_mask = prompt_attention_mask.eq(0)
        
        # Run through encoder
        memory = self.transformer.encoder(
            src.transpose(0, 1),
            src_key_padding_mask=src_key_padding_mask
        )
        
        return memory, src_key_padding_mask
    
    def decode(self, tgt, memory, src_key_padding_mask):
        """Decode the target sequence using the encoder memory."""
        # Embed target tokens
        tgt_emb = self.chord_embedding(tgt)
        
        # Add positional encoding
        tgt_emb = self.pos_encoder(tgt_emb)
        
        # Create target mask (causal/autoregressive)
        tgt_len = tgt.size(1)
        tgt_mask = torch.triu(
            torch.ones(tgt_len, tgt_len, device=tgt.device) * float('-inf'), 
            diagonal=1
        )
        
        # Create target padding mask
        tgt_key_padding_mask = tgt.eq(0)
        
        # Run through decoder
        decoder_output = self.transformer.decoder(
            tgt_emb.transpose(0, 1),
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask
        )
        
        # Project to vocabulary
        logits = self.output_projection(decoder_output.transpose(0, 1))
        
        return logits
    
    def forward(self, prompt_ids, prompt_attention_mask, chord_tokens):
        """Full forward pass."""
        # Encode prompt
        memory, src_key_padding_mask = self.encode(prompt_ids, prompt_attention_mask)
        
        # Use teacher forcing for training
        # Shift right to create decoder input (remove last token, add BOS at beginning)
        tgt_input = chord_tokens[:, :-1]
        
        # Decode
        logits = self.decode(tgt_input, memory, src_key_padding_mask)
        
        # Prepare target (shift left: remove first token which is BOS)
        target = chord_tokens[:, 1:]
        
        return logits, target
    
    def generate(self, prompt_ids, prompt_attention_mask, max_length=50, 
                 temperature=1.0, top_k=0, top_p=0.9):
        """Generate a chord progression from lyrics."""
        batch_size = prompt_ids.size(0)
        device = prompt_ids.device
        
        # Encode prompt
        memory, src_key_padding_mask = self.encode(prompt_ids, prompt_attention_mask)
        
        # Start with BOS token
        current_token = torch.full((batch_size, 1), 1, dtype=torch.long, device=device)
        
        # Storage for generated tokens
        generated = [current_token]
        
        # Generation loop
        for _ in range(max_length - 1):
            # Decode current sequence
            logits = self.decode(current_token, memory, src_key_padding_mask)
            
            # Get the next token probabilities
            next_token_logits = logits[:, -1, :] / temperature
            
            # Apply top-k filtering
            if top_k > 0:
                top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k, dim=-1)
                next_token_logits = torch.zeros_like(next_token_logits).scatter_(-1, top_k_indices, top_k_logits)
            
            # Apply top-p (nucleus) filtering
            if top_p < 1.0:
                sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True, dim=-1)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                
                # Remove tokens with cumulative probability above the threshold
                sorted_indices_to_remove = cumulative_probs > top_p
                # Shift the indices to the right to keep the first token above threshold
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                sorted_indices_to_remove[..., 0] = 0
                
                # Create a mask for indices to remove
                indices_to_remove = sorted_indices_to_remove.scatter(
                    -1, sorted_indices, sorted_indices_to_remove
                )
                next_token_logits[indices_to_remove] = -float('inf')
            
            # Sample from the filtered distribution
            probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Add to generated
            generated.append(next_token)
            current_token = torch.cat(generated, dim=1)
            
            # Stop if EOS token is generated
            if (next_token == 2).any():
                break
        
        # Concatenate all tokens
        result = torch.cat(generated, dim=1)
        
        return result

In [19]:
# Training implementation
def train_model(df, batch_size=32, epochs=10, learning_rate=1e-4):
    """Train the lyrics-to-chord progression model."""
    # Split dataset
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
    print(f"Training with {len(train_df)} examples, validating with {len(val_df)} examples")
    
    # Initialize tokenizers
    text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    chord_tokenizer = ChordTokenizer()
    
    # Build chord vocabulary from training data
    chord_tokenizer.fit(train_df['roman_chords'].dropna())
    
    # Create datasets
    train_dataset = LyricsChordDataset(train_df, text_tokenizer, chord_tokenizer)
    val_dataset = LyricsChordDataset(val_df, text_tokenizer, chord_tokenizer)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LyricsToChordModel(chord_vocab_size=len(chord_tokenizer.chord_to_id))
    model.to(device)
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Initialize scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=2, verbose=True
    )
    
    # Training loop
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            # Move batch to device
            prompt_ids = batch["prompt_ids"].to(device)
            prompt_attention_mask = batch["prompt_attention_mask"].to(device)
            chord_tokens = batch["chord_tokens"].to(device)
            
            # Forward pass
            logits, target = model(prompt_ids, prompt_attention_mask, chord_tokens)
            
            # Calculate loss
            loss = F.cross_entropy(
                logits.reshape(-1, model.chord_vocab_size),
                target.reshape(-1),
                ignore_index=0  # Ignore padding
            )
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                # Move batch to device
                prompt_ids = batch["prompt_ids"].to(device)
                prompt_attention_mask = batch["prompt_attention_mask"].to(device)
                chord_tokens = batch["chord_tokens"].to(device)
                
                # Forward pass
                logits, target = model(prompt_ids, prompt_attention_mask, chord_tokens)
                
                # Calculate loss
                loss = F.cross_entropy(
                    logits.reshape(-1, model.chord_vocab_size),
                    target.reshape(-1),
                    ignore_index=0  # Ignore padding
                )
                
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        # Update scheduler
        scheduler.step(val_loss)
        
        # Log metrics
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "chord_progression_model_2.pt")
            print(f"Saved new best model with val loss: {val_loss:.4f}")
    
    return model, chord_tokenizer

In [18]:
# Inference function
def generate_chord_progression_from_lyrics(lyrics, model, text_tokenizer, chord_tokenizer, max_length=10):
    """Generate a chord progression from lyrics."""
    device = next(model.parameters()).device
    
    # Tokenize lyrics
    lyrics_encoding = text_tokenizer(
        lyrics,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    
    # Move to device
    prompt_ids = lyrics_encoding["input_ids"].to(device)
    prompt_attention_mask = lyrics_encoding["attention_mask"].to(device)
    
    # Generate
    model.eval()
    with torch.no_grad():
        generated = model.generate(
            prompt_ids, 
            prompt_attention_mask,
            max_length=max_length
        )
    
    # Decode
    chord_progression = chord_tokenizer.decode(generated[0].cpu().numpy())
    
    return chord_progression

In [31]:
#model, chord_tokenizer = train_model(df, batch_size=32, epochs=5)


In [ ]:

def quick_model_load(df=df, model_path='chord_progression_model.pt', random_state=42):
    
    """Loads a model that has already been trained, just be careful to use the same random_state value as the one in the test training split."""
    
    chord_tokenizer = ChordTokenizer()
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=random_state)

    chord_tokenizer.fit(train_df['roman_chords'].dropna())
    model = LyricsToChordModel(chord_vocab_size=len(chord_tokenizer.chord_to_id))

    model.load_state_dict(torch.load(model_path,weights_only=True))
    return model, chord_tokenizer


model, chord_tokenizer = quick_model_load()

Created chord vocabulary with 5638 tokens


/home/mario/.pyenv/versions/lewagon/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mario/.pyenv/versions/lewagon/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [30]:
# Test the model with a sample lyric
sample_lyric = "In the darkness of the night, I feel alone and lost"
text_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Generate chord progression
progression = generate_chord_progression_from_lyrics(
    sample_lyric, model, text_tokenizer, chord_tokenizer
)

print(f"Lyrics: {sample_lyric}")
print(f"Generated chord progression: {progression}")

Lyrics: In the darkness of the night, I feel alone and lost
Generated chord progression: imin bVII bIII v imin v bIII bVII bIII
